# Stock Market Volatility Analysis

### Introduction: Suppose the volatility of the stock market is governed by a hidden factor that is not directly measurable. We attempt to establish a Hidden Markov Model to quantify how the volatility of a particular stock is influenced by that mysterious ecomomical factor. Moreover, we quantify if the stock in consideration is of high volatility or of low volatility.
### Data: The data we use is the first 2200 historical records of the open prices of the Carriage Service Inc.
### Model: A Hidden Markov Model with three hidden states: bear market, normal market, bull market. We measure volatility by one-day percentage price change. We assume conditioning on each hidden state, the one-day percentage price change follows a normal distribution whose mean and variance are to be determined such that the probability of seeing the actual open price sequence is maximized. A similar (if not the same) approach is used in  http://jcyhong.github.io/assets/intro-hmm-stock.pdf where the means conditioning on different hidden states are assumed to be zero. We choose not to use solely variance to describe the volatility since we want to know if the price is likely to grow up or down. The variance alone fails to provide the up/down infomation. Mathematically, two mean-zero normal random variables $X$ and -$X$ share the same variance but have opposite signs.
### Validation Method: We will use a training set to fit our model and a testing set to see if our model describes the pattern in the testing set.
### Notations: In most context of $\pi$ denotes the initial state distribution of the hidden states; A denotes the transition matrix of the hidden states; B is the aquisition matrix storing the parameters  of the distribution of observable events conditioning on different hidden states. We will use the same notations. 
### Limitation: The Hidden Markov Model developed in this project is not meaned for stock price prediction as that will give poor prediction results (see http://jcyhong.github.io/assets/intro-hmm-stock.pdf for more).


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
from scipy.stats import norm


### Load data and generate the one-day price change: (Price in Day $i+1$ -Price in Day $i$)/Price in Day $i$. Separate the price change sequence into a training set and a testing set with a ratio  about 10:1 .

In [0]:
CarriageService = pd.read_csv('gdrive/My Drive/CarriageServicesStockPrice.csv')
CarriageServiceTrain = CarriageService.iloc[:2000,:][['Open']]
CarriageServiceTest = CarriageService.iloc[2000:2200,:][['Open']]

In [0]:
price_sequence_train = np.array(CarriageServiceTrain).reshape(1,-1)[0]

price_sequence_test = np.array(CarriageServiceTest).reshape(1,-1)[0]

print(len(price_sequence_train),len(price_sequence_test))

2000 200


In [0]:
train_diff = np.diff(price_sequence_train)
test_diff = np.diff(price_sequence_test)
train_diff = train_diff/price_sequence_train[0:len(price_sequence_train)-1]
test_diff = test_diff/price_sequence_test[0:len(price_sequence_test)-1]

#### Stats on the difference data.



In [0]:
np.mean(train_diff)

-0.001155761181186319

In [0]:
np.max(train_diff)

0.19402985074626866

In [0]:
np.min(train_diff)

-0.2025862068965517

In [0]:
np.std(train_diff)

0.03275022339150875

In [0]:
train_diff[-1]

-0.045454545454545456

### Initialization of Pi, A, B. 






In [0]:
Pi = np.array([0.33,0.33,0.34])
A = np.array([[0.33,0.33,0.34],[0.33,0.33,0.34],[0.33,0.33,0.34]]) #A[i][j] = P(X_{t}=j|X_{t-1}=i)
B = np.array([[-0.20,0,0.20],[0.033,0.033,0.033]]).T

### The Alpha-pass function

#### Calculate $P(X \in B_{\epsilon}(pricediff))$ given that X is a normal with mean row[0] and standard deviation row[1].

In [0]:
def local_prob(price_diff, epsilon, param_matrix):    
    return [norm.cdf((price_diff+epsilon-row[0])/row[1])-norm.cdf((price_diff-epsilon-row[0])/row[1]) for index, row in enumerate(param_matrix)]
        


$\alpha_{t}(i) = P(B_{\epsilon}(O_{0}),...,B_{\epsilon}(O_{t}),x_{t} = q_{i}| \lambda)$

$\alpha_{0}(i) = \pi_{i} b_{i}(O_{0})$

$\alpha_{t}(i) = \sum_{j} \alpha_{t-1}(j)a_{ji}b_{i}(B_{\epsilon}(O_{t}))$

In [0]:
def forward_pass(Pi, A, B): 
    epsilon = 0.2
    # alpha_{0}    
    c = [1/np.dot(local_prob(train_diff[0],epsilon,B),Pi)]
    alpha = [np.array([item[0]*item[1]*c[0] for index, item in enumerate(np.vstack([Pi,local_prob(train_diff[0],epsilon,B)]).T)])]
    # alpha_{t} t = 1,...,T-1
    for t in range(1,len(train_diff)): # initialization for the rest in the price sequence
        c += [0]
        alpha.append(np.zeros(shape=A.shape[1]))
    for t in range(1,len(train_diff)):             
        for i in range(A.shape[1]):            
            alpha[t][i] += np.dot(alpha[t-1],A.T[i])*local_prob(train_diff[t],epsilon,B)[i] 
        c[t] = 1/np.sum([alpha[t][i] for i in range(A.shape[1])])    
        for i in range(A.shape[1]):
            alpha[t][i] *= c[t]
    return [alpha,c]

#forward_pass(Pi,A,B)[0] # this gives alpha, uncomment to test
#forward_pass(Pi,A,B)[1] # this gives c, uncomment to test

### The Beta-pass function

$\beta_{t}(i) = P(B_{\epsilon}(O_{t+1}),...,B_{\epsilon}(O_{T-1})|x_{t}=q_{i},\lambda)$

$\beta_{T-1}(i)=1$

$\beta_{t}(i) = \sum_{j}a_{ij}b_{j}(B_{\epsilon}(O_{t+1}))\beta_{t+1}(j)$

In [0]:
def backward_pass(Pi, A, B):
    epsilon = 0.2
    # initialization
    beta = []
    for t in range(0,len(train_diff)):
        beta.append(np.zeros(A.shape[1]))
    c = forward_pass(Pi,A,B)[1]
    # beta_{T-1}
    beta[len(train_diff)-1]=c[len(train_diff)-1]*np.ones(shape=A.shape[1])
    
    # beta_{t} t = T-2, ..., 0
    for t in range(len(train_diff)-2,-1,-1):
        for i in range(A.shape[1]):
            for j in range(A.shape[1]):
                beta[t][i] += A[i][j]*local_prob(train_diff[t+1],epsilon,B)[j]*beta[t+1][j]
            beta[t][i] *= c[t] # beta_hat
    return beta

#backward_pass(Pi, A, B) # uncomment to test

### The gamma_functions returns di-gamma(denoted by Gamma) and gamma

Gamma: $\gamma_{t}(i)=P(x_{t}=q_{i}|O,\lambda) = \sum_{j}\gamma_{t}(i,j)$

gamma: $\gamma_{t}(i,j) = P(x_{t}=q_{i},x_{t+1}=q_{j}|O,\lambda) = \alpha_{t}(i)a_{ij}b_{j}(B_{\epsilon}(O_{t+1}))\beta_{t+1}(j)$

Please note that $\alpha$ and $\beta$ are scaled so no need to divide by $P(O|\lambda)$ in $\gamma_{t}(i,j)$.

In [0]:
def gamma_functions(Pi, A, B):
    epsilon = 0.2
    # initialization 
    Gamma = [] # Gamma_{t}(i) = P(x_{t}=q_{i}|O,lambda) for t = 0,...,T-1; i=0,1
    gamma = [] # gamma_{t}(i,j) = P(x_{t}=q_{i},x_{t+1}=q_{j}|O,lambda) for t = 0,...,T-1; i,j=0,1
    alpha = forward_pass(Pi, A, B)[0]
    beta = backward_pass(Pi, A, B)
    for t in range(0,len(train_diff)):
        Gamma.append(np.zeros(shape=A.shape[1]))
        gamma.append(np.zeros(shape=(A.shape[1],A.shape[1])))
    
    # Real computation begins
    for t in range(0,len(train_diff)-1):
        for i in range(A.shape[1]):
            for j in range(A.shape[1]):
                gamma[t][i][j] += alpha[t][i]*A[i][j]*local_prob(train_diff[t+1],epsilon,B)[j]*beta[t+1][j]
                Gamma[t][i] += gamma[t][i][j]

    # Gamma_{T-1} is special and cannot be generated by induction
    Gamma[len(train_diff)-1] = alpha[len(train_diff)-1]
    
    return [Gamma, gamma]

#gamma_functions(Pi,A,B)[0] # this gives Gamma, uncomment to test
#gamma_functions(Pi,A,B)[1] # this gives gamma, uncomment to test

### Update Pi, A and B based on 1 round of iteration

#### Auxillary function to measure $P(X \in B_{\epsilon}(pricediff) | \lambda)$ , $X$ is normal with mean and variance prescribed by $\lambda$.

In [0]:
def local_prob_interval(price_diff, epsilon, param_matrix):    
    return np.array([np.array([norm.cdf((price_diff-epsilon-row[0])/row[1]),norm.cdf((price_diff+epsilon-row[0])/row[1])])for index, row in enumerate(param_matrix)])
#local_prob_interval(5, 0.01, B)

$\pi = \gamma_{0}(i)$

$a_{ij} = \sum_{t=0}^{T-2} \gamma_{t}(i,j)/\sum_{t=0}^{T-2} \gamma_{t}(i)$

$b_{j}(B_{\epsilon}(k)) = \sum_{t:O_{t} \in B_{\epsilon}(k)} \gamma_{t}(i) /\sum_{t} \gamma_{t}(j)$ 

#### In the following, k =price_diff. Recall that we want to maximize the likelihood of the observed price_diff sequence.


In [0]:
def model_update(Pi, A, B):
    epsilon = 0.2
    price_diff_sequence = train_diff
    
    # Initialization
    A_new = np.zeros(A.shape)
    B_new = np.zeros(B.shape)
    prob_transit_interval = [local_prob_interval(price_diff, epsilon, B) for index, price_diff in enumerate(price_diff_sequence)]
    [Gamma, gamma] = gamma_functions(Pi, A, B)
    
    # Update Pi
    Pi_new = Gamma[0]                                             
    
    # Update A
    for i in range(A.shape[0]):
        denom = 0
        for t in range(len(price_diff_sequence)-1):
            denom += Gamma[t][i]
        for j in range(A.shape[1]):
            numer = 0
            for t in range(len(price_diff_sequence)-1):
                numer += gamma[t][i][j]
            A_new[i][j] = numer/denom
    
    # Update B
    for j in range(A.shape[0]):
        for index, item in enumerate(prob_transit_interval):
            numer = 0
            for t in range(len(price_diff_sequence)):   
                if norm.cdf((price_diff_sequence[t]-B[j][0])/B[j][1]) <= item[j][1] and norm.cdf((price_diff_sequence[t]-B[j][0])/B[j][1]) >= item[j][0]:
                   numer += Gamma[t][j]

            denom = 0
            for t in range(len(price_diff_sequence)):
                denom += Gamma[t][j]

            prob_transit = numer/denom
            
            mu_sigma_pair = []
            for index1, item1 in enumerate(np.arange(-0.2+0.133*j,-0.066+0.133*j,0.02)):
                for index2, item2 in enumerate(np.arange(0.01,0.06,0.02)):
                    mu_sigma_pair.append([item1,item2])
                    #print(mu_sigma_pair)
            #best_mu_index = np.argmin([np.abs(norm.cdf((price_sequence[t]+epsilon-mu)/B[j][1])\
            #                                  - norm.cdf((price_sequence[t]-epsilon-mu)/B[j][1]) \
            #                                  - prob_transit) for mu in np.arange(B[j][0]-2,B[j][0]+2,0.5)])
            
            #best_sigma_index = np.argmin([np.abs(norm.cdf((price_diff_sequence[t]+epsilon-B[j][0])/sigma)\
            #                                  - norm.cdf((price_diff_sequence[t]-epsilon-B[j][0])/sigma) \
            #                                  - prob_transit) for sigma in np.arange(0.1+0.1*j,0.11+0.1*j,0.02)])
            best_index = np.argmin([np.abs(norm.cdf((price_diff_sequence[t]+epsilon-item[0])/item[1])\
                                              - norm.cdf((price_diff_sequence[t]-epsilon-item[0])/item[1]) \
                                              - prob_transit) for index, item in enumerate(mu_sigma_pair)])
            #print(best_index)
        B_new[j][0] = mu_sigma_pair[best_index][0] # the mean is updated
        #B_new[j][1] = B[j][1] # the deviation remains unchanged
        B_new[j][1] = mu_sigma_pair[best_index][1] # the deviation is updated
        #B_new[j][0] = B[j][0] # the mean remains unchanged
        #B_new[j][1] = np.arange(0.01,0.6,0.1)[best_sigma_index]
      
    return [Pi_new, A_new, B_new]

#model_update(Pi,A,B) 

### Calculate the log of P(O|lambda). We will denote it by log probability in the following contexts.

$1= \sum_{j}\hat{\alpha}_{T-1}(j)=c_{0}\dots c_{T-1}P(O|\lambda)$ 

$\log P(O|\lambda) = -\sum_{i=1}^{T-1}\log c_{i}$

In [0]:
def LogProb(Pi,A,B):
    c = forward_pass(Pi,A,B)[1]
    logProb = np.sum([-np.log(item) for index, item in enumerate(c)])
    return logProb
LogProb(Pi,A,B) #uncomment to test

-408.4546168935725

### Return Pi, A, B and log_prob based on many iterations.







In [0]:
def improvement_global(Pi,A,B,max_iters=10):
    # initialization
    print("LogProb was ", LogProb(Pi,A,B), " initially.")
    print("____________________________________________________")
    params = []
    
    # iteration
    for rounds in range(1,max_iters+1):         
        print("Round %d starts." % rounds)        
        [Pi, A, B] =  [model_update(Pi,A,B)[0],model_update(Pi,A,B)[1],model_update(Pi,A,B)[2]] 
        Log = LogProb(Pi, A, B)
        print([Pi,A,B,Log])
        params.append([Pi,A,B,Log]) # The initial is not attached to params.
        print("The log probability after this round of updating is %.3f " % Log)
        print("____________________________________________________")
        
    # store results
    best_index = np.argmax(np.array([item[3] for index, item in enumerate(params)]))
    best_params = params[best_index]
    print(best_params)
    
    #return best_params
    return best_params


In [0]:
import time
t = time.time()
best_params = improvement_global(Pi,A,B,5)
elapsed = time.time()-t
print("Elapsed time is %.3f seconds." % elapsed)

LogProb was  -408.4546168935725  initially.
____________________________________________________
Round 1 starts.
[array([4.43817322e-07, 4.65973487e-01, 5.34026069e-01]), array([[0.25395541, 0.49589747, 0.25014712],
       [0.25586962, 0.49599283, 0.24813755],
       [0.25816131, 0.49609892, 0.24573977]]), array([[-0.16 ,  0.01 ],
       [ 0.013,  0.05 ],
       [ 0.086,  0.03 ]]), -28.046388320244578]
The log probability after this round of updating is -28.046 
____________________________________________________
Round 2 starts.
[array([0.        , 0.43006831, 0.56993169]), array([[0.2361371 , 0.508847  , 0.2550159 ],
       [0.236358  , 0.51010273, 0.25353927],
       [0.23875502, 0.51015614, 0.25108884]]), array([[-0.16 ,  0.01 ],
       [ 0.013,  0.05 ],
       [ 0.126,  0.01 ]]), -30.18905608824851]
The log probability after this round of updating is -30.189 
____________________________________________________
Round 3 starts.
[array([0.       , 0.3948815, 0.6051185]), array([[0.2

### It seems like 

1.   under the bear market, the stock price will decrease 16% on a daily basis, with 1% of variance in the percentage change;
2.   under the normal market, the stock price will increase 1.3% on a daily basis, with 5% of variance in the percentage change;
3.   under the bull market, the stock price will decrease 12.6% on a daily basis, with 1% of variance in the percentage change.
4.   once in a bear or bull market, we will move back to a normal market with higher probability instead of staying in the same market. 
5.   overall this stock is growing with a steady pace.

#### Note that under some market policies, the stock price can only variate from -10% to 10%. More work needs to be done for our model to be applied in a real market! Assume for simplicity that our stock price does not follow this market rule, let us see how well our model fits the real test data. 





### Classification of the hidden states. Choose the hidden state to be the one such that its governing normal distribution mean is closest to the actual price growth (difference).

In [0]:
def detect_state(price_diff):
    return np.argmin([np.abs(price_diff - item) for index, item in enumerate([-0.16,0.013,0.126])])
    
test_state = [detect_state(item) for index, item in enumerate(test_diff)]

### Classification of the hidden states on the test data. Each row of the resulting matrix counts the number of each state in the next day provided that the state of the current day is known. For example, in row 2, the current day is in a normal market, we see 13 cases of bear market, 140 cases of normal market, 11 cases of bull market in the next day.

In [0]:
Count_00 = 0
Count_01 = 0
Count_02 = 0
Count_10 = 0
Count_11 = 0
Count_12 = 0
Count_20 = 0
Count_21 = 0
Count_22 = 0
for index, item in enumerate([[test_state[i], test_state[i+1]] for i in range(len(test_state)-1)]):
    if item[0] == 0:
       if item[1] == 0:
          Count_00 += 1
       elif item[1] == 1:
            Count_01 += 1
       else:
            Count_02 += 1
    elif item[0] == 1:
       if item[1] == 0:
          Count_10 += 1
       elif item[1] == 1:
            Count_11 += 1
       else:
            Count_12 += 1
    else:
       if item[1] == 0:
          Count_20 += 1
       elif item[1] == 1:
            Count_21 += 1
       else:
            Count_22 += 1
          
np.array([[Count_00,Count_01,Count_02],[Count_10,Count_11,Count_12],[Count_20,Count_21,Count_22]])
    

array([[  0,  11,   4],
       [ 13, 140,  11],
       [  2,  12,   5]])

### The result on the test set validates that the stock is at most time in a normal market.

### Directions of improvement: 

Choose the number of states more carefully. We chose three in this project for a demonstration. We can do better with more effort. A larger number of hidden states, and a finer partition of the growth rate would very likely to give more insightful volatility information.


